In [1]:
#!pip install konlpy
!pip install --upgrade wandb

In [2]:
import os
import pandas as pd
import tensorflow
import numpy as np
import transformers
import torch
import pandas as pd
import tensorflow as tf

from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

print(transformers.__version__)
print(tf.__version__)



4.51.3
2.18.0


In [3]:
import tensorflow as tf
import os

if 'COLAB_TPU_ADDR' in os.environ:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
        tpu='grpc://' + os.environ['COLAB_TPU_ADDR']
    )
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("TPU initialized.")
else:
    print("TPU 환경이 아님. GPU 또는 CPU 사용 중입니다.")

TPU 환경이 아님. GPU 또는 CPU 사용 중입니다.


**데이타**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC

/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC


original_train 데이타만 사용한다.

In [6]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir)
train_data_path = os.path.join(data_dir, 'merged_train.csv')

train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,1951,직장 내 괴롭힘 대화,팀장님 이거 언제까지 마무리 하면 될까요?\n무리하지 말고 넉넉하게 주말까지 다 작...
1,4756,일반 대화,"내일 날씨 어떻대?\n비 온다던데. 우산 챙겨가야 할 것 같아.\n에이, 야외 활동..."
2,1234,기타 괴롭힘 대화,야 쟤 좀 봐.\n 꼴에 유행하는 옷 입었네 \n 호박에 줄 긋는다고 수박되나 \n...
3,4767,일반 대화,오늘 수업 내용 이해했어?\n솔직히 좀 어려웠어. 너는?\n나도 몇 부분이 헷갈리더...
4,1511,직장 내 괴롭힘 대화,자네 내일 아침에 우리집 들려서 출근하게\n네?\n내가 차가 고장났어\n아. 그런데...


데이터 전처리

In [7]:
print('전체 샘플수 :', (len(train_data)))
train_data.info()

전체 샘플수 : 4637
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4637 entries, 0 to 4636
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           4637 non-null   int64 
 1   class         4637 non-null   object
 2   conversation  4637 non-null   object
dtypes: int64(1), object(2)
memory usage: 108.8+ KB


In [8]:
# 클래스 라벨을 숫자로 변환
# 0, 1, 2, 3, 4
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']


class_dict = {}
for idx, class_name in enumerate(CLASS_NAMES):
    class_dict[class_name] = idx

train_data['label'] = train_data['class'].apply(lambda s: class_dict[s])

train_data.head()

,idx,class,conversation,label
0,1951,직장 내 괴롭힘 대화,팀장님 이거 언제까지 마무리 하면 될까요?\n무리하지 말고 넉넉하게 주말까지 다 작...,2
1,4756,일반 대화,"내일 날씨 어떻대?\n비 온다던데. 우산 챙겨가야 할 것 같아.\n에이, 야외 활동...",4
2,1234,기타 괴롭힘 대화,야 쟤 좀 봐.\n 꼴에 유행하는 옷 입었네 \n 호박에 줄 긋는다고 수박되나 \n...,3
3,4767,일반 대화,오늘 수업 내용 이해했어?\n솔직히 좀 어려웠어. 너는?\n나도 몇 부분이 헷갈리더...,4
4,1511,직장 내 괴롭힘 대화,자네 내일 아침에 우리집 들려서 출근하게\n네?\n내가 차가 고장났어\n아. 그런데...,2


In [9]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣ㅏ-ㅣ?.!,]+", " ", sentence)

    sentence = sentence.strip()

    return sentence

In [10]:
train_data["conversation"] = [preprocess_sentence(s) for s in train_data["conversation"]]
train_data.head()

,idx,class,conversation,label
0,1951,직장 내 괴롭힘 대화,팀장님 이거 언제까지 마무리 하면 될까요 ? 무리하지 말고 넉넉하게 주말까지 다 작...,2
1,4756,일반 대화,"내일 날씨 어떻대 ? 비 온다던데 . 우산 챙겨가야 할 것 같아 . 에이 , 야외 ...",4
2,1234,기타 괴롭힘 대화,야 쟤 좀 봐 . 꼴에 유행하는 옷 입었네 호박에 줄 긋는다고 수박되나 으엑 . 못...,3
3,4767,일반 대화,오늘 수업 내용 이해했어 ? 솔직히 좀 어려웠어 . 너는 ? 나도 몇 부분이 헷갈리...,4
4,1511,직장 내 괴롭힘 대화,자네 내일 아침에 우리집 들려서 출근하게 네 ? 내가 차가 고장났어 아 . 그런데 ...,2


**test data**

In [11]:
data_dir = os.path.join(current_dir)
test_data_path = os.path.join(data_dir, 'test.csv')


test_data = pd.read_csv(test_data_path)
test_data.head()

,idx,text
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...


In [12]:
# 한 문장의 길이를  num_tokens에 저장
num_tokens = [len(tokens) for tokens in train_data['conversation']]
# array로 변환
num_tokens = np.array(num_tokens)

# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다.
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('train_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 maxlen 설정값 이내에 포함됩니다. ')

sequences_maxlen = maxlen

sequences_maxlen = 300


문장길이 평균 :  230.5475522967436
문장길이 최대 :  909
문장길이 표준편차 :  108.46577725734618
train_sequences maxlen :  447
전체 문장의 0.9566530084106103%가 maxlen 설정값 이내에 포함됩니다. 


In [13]:
# 한 문장의 길이를  num_tokens에 저장
num_tokens = [len(tokens) for tokens in test_data['text']]
# array로 변환
num_tokens = np.array(num_tokens)

# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다.
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

max_tokens = np.mean(num_tokens) + 2.5 * np.std(num_tokens)
maxlen = int(max_tokens)
print('test_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 maxlen 설정값 이내에 포함됩니다. ')

test_maxlen = maxlen

문장길이 평균 :  213.626
문장길이 최대 :  722
문장길이 표준편차 :  101.13918194250931
test_sequences maxlen :  466
전체 문장의 0.972%가 maxlen 설정값 이내에 포함됩니다. 


훈련/검증 데이터셋 분리
- validation set은 original_train_set으로 부터 생성하도록 고정
- random_state=42

In [14]:
# train/ val 분리
random_state=42

df = train_data

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['conversation'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=random_state,
    stratify=df['label']
)

**beomi/kcbert-base**

In [15]:
from transformers import TFBertForSequenceClassification

model_name = "beomi/kcbert-base"

# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained(model_name)

#PyTorch로 학습된 beomi/kcbert-base 모델을 TensorFlow로 변환하여 사용
from transformers import TFBertModel

model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=len(class_dict),  from_pt=True)#  클래스 수에 맞게

# tokenizer의 vocab 사이즈 확인
tokenizer.vocab_size


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


30000

In [16]:
# TensorFlow 데이터셋으로 변환
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=sequences_maxlen)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=sequences_maxlen)

# TensorFlow 데이터셋으로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

# 배치 처리 및 셔플
train_dataset = train_dataset.shuffle(1000).batch(16)
val_dataset = val_dataset.shuffle(1000).batch(16)

In [17]:
from transformers import AutoConfig


config = AutoConfig.from_pretrained(model_name)
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.51.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

In [18]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU 사용 가능 여부 확인
    print("TPU detected:", tpu.cluster_spec().as_dict())
except ValueError:
    print("TPU not detected. Running on CPU/GPU.")

TPU not detected. Running on CPU/GPU.


In [19]:
strategy = tf.distribute.get_strategy()
print("Current strategy:", type(strategy).__name__)

Current strategy: _DefaultDistributionStrategy


In [20]:

# 👇 dummy input으로 모델 빌드 (이게 중요!)
#model(tf.constant([[1]*128]))

In [21]:
# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])



In [22]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108918528 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 108922373 (415.51 MB)
Trainable params: 108922373 (415.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
import wandb
from wandb.integration.keras.callbacks import WandbMetricsLogger
wandb.login()

# W&B 프로젝트 초기화
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="jiwoong-team",
    # Set the wandb project where this run will be logged.
    project="DLThon-DKTC",
    # Track hyperparameters and run metadata.
    name=f"moya_KcBert300_trainonly", # 실험 이름
    notes="제공된 train 데이터를 사용한 기준선 실험", # 실험에 대한 간단한 설명
    config={ # 세부 구성 내용
        "experiment_name": "kcbert",
        "general_conversation_type": "없음", # 일반 데이터 타입
        "architecture": "kcbert-base)",
    },
)



wandb: Currently logged in as: moya9292 (jiwoong-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [24]:
# 모델 학습
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
    #callbacks=[WandbMetricsLogger()]  # WandbMetricsLogger를 콜백으로 넘겨주기만 하면 알아서 epoch별 metric을 기록
)

Epoch 1/3
232/232 [==============================] - 315s 1s/step - loss: 0.4588 - accuracy: 0.8452 - val_loss: 0.2944 - val_accuracy: 0.9073
Epoch 2/3
232/232 [==============================] - 261s 1s/step - loss: 0.1410 - accuracy: 0.9550 - val_loss: 0.2717 - val_accuracy: 0.9138
Epoch 3/3
232/232 [==============================] - 260s 1s/step - loss: 0.0637 - accuracy: 0.9811 - val_loss: 0.3525 - val_accuracy: 0.9041


In [25]:
from sklearn.metrics import classification_report
# 최적의 모델로 검증세트 예측


# 예측 수행
logits = model.predict(val_dataset).logits
# 소프트맥스 후 argmax로 클래스 인덱스 추출
y_pred_labels = np.argmax(logits, axis=-1)


# val_dataset에서 정답 라벨만 추출
y_true_labels = []

for batch in val_dataset:
    _, labels = batch  # unpack (inputs, labels)
    y_true_labels.extend(labels.numpy())

y_true_labels = np.array(y_true_labels)


# wandb Table 생성
report = classification_report(y_true_labels, y_pred_labels, target_names=CLASS_NAMES, output_dict=True)
columns = ["class", "precision", "recall", "f1-score", "support"]
data = []

for label, metrics in report.items():
    if isinstance(metrics, dict):  # dict일 때만 values 가져오기
        row = [label] + [metrics.get(col, None) for col in columns[1:]]
        data.append(row)

table = wandb.Table(columns=columns, data=data)

# wandb에 훈련 결과 기록
wandb.log({
    "classification_report_table": table,
    "macro_f1": report["macro avg"]["f1-score"],
    "accuracy": report["accuracy"],
    "threat_f1": report["협박 대화"]["f1-score"],
    "extortion_f1": report["갈취 대화"]["f1-score"],
    "workplace_bullying_f1": report["직장 내 괴롭힘 대화"]["f1-score"],
    "other bullying": report["기타 괴롭힘 대화"]["f1-score"],
    "general_conversation_f1": report["일반 대화"]["f1-score"],
})

# 실험 종료
run.finish()

58/58 [==============================] - 25s 379ms/step


accuracy,▁
extortion_f1,▁
general_conversation_f1,▁
macro_f1,▁
other bullying,▁
threat_f1,▁
workplace_bullying_f1,▁
accuracy,0.17241
extortion_f1,0.16
general_conversation_f1,0.19048
macro_f1,0.17188


Test 데이타 예측

In [26]:

def predict_labels(model, tokenizer, texts, max_length=128, batch_size=16):
    """
    주어진 텍스트 리스트에 대해 예측된 클래스 인덱스를 반환하는 함수.

    Args:
        model: 학습된 TensorFlow 모델
        tokenizer: Hugging Face tokenizer
        texts (List[str]): 예측할 텍스트 목록
        max_length (int): 입력 시퀀스 최대 길이
        batch_size (int): 배치 사이즈

    Returns:
        np.ndarray: 예측된 클래스 인덱스 배열
    """
    # 텍스트 인코딩
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)

    # TensorFlow Dataset 생성
    dataset = tf.data.Dataset.from_tensor_slices(dict(encodings)).batch(batch_size)

    # 예측 수행
    logits = model.predict(dataset).logits

    # 소프트맥스 후 argmax로 클래스 인덱스 추출
    predictions = np.argmax(logits, axis=-1)

    return predictions


In [27]:
# 예측
test_texts = test_data["text"].tolist()
predictions = predict_labels(model, tokenizer, test_texts, max_length=sequences_maxlen)
print(predictions)

32/32 [==============================] - 15s 368ms/step
[1 2 2 3 3 0 0 1 2 1 0 3 2 2 2 0 3 2 0 3 0 3 3 1 1 2 3 3 2 2 1 0 3 1 3 3 3
 1 0 1 0 1 2 1 3 2 0 3 3 0 1 3 1 2 2 3 3 3 3 3 1 3 0 3 2 0 3 3 4 1 3 2 3 2
 0 2 2 1 2 3 1 0 4 1 0 2 1 3 3 2 2 2 0 1 3 2 1 2 0 2 1 1 2 0 3 2 2 3 3 0 1
 0 1 2 1 1 3 2 2 1 2 3 3 1 1 1 2 1 1 3 3 3 2 3 1 0 3 0 3 2 0 3 1 3 2 0 0 0
 0 1 0 3 4 1 2 3 0 2 3 3 0 3 4 0 2 3 2 2 3 1 0 2 2 3 2 3 2 2 1 1 2 3 2 2 0
 3 2 0 1 3 0 3 1 3 1 3 0 1 0 3 2 0 3 3 0 3 2 1 2 2 0 1 3 2 1 3 3 3 3 0 1 4
 0 3 0 3 1 0 1 0 2 0 3 3 1 3 3 1 0 3 3 3 3 2 2 1 2 1 2 1 0 2 0 2 2 1 3 2 1
 2 1 3 1 3 2 3 3 3 2 1 0 3 0 2 1 2 2 3 2 2 1 0 0 2 2 2 1 2 1 3 1 1 3 3 2 3
 3 3 1 0 2 3 0 2 1 1 3 2 1 2 3 1 3 3 2 3 0 3 3 0 2 0 0 1 1 1 1 3 2 3 1 0 0
 3 1 2 3 1 3 3 0 2 0 0 1 2 2 1 0 0 0 3 1 1 1 2 3 3 1 1 3 3 1 1 2 2 3 2 0 1
 2 2 0 2 2 3 1 1 1 1 2 0 2 2 3 1 3 0 0 1 1 0 1 2 1 3 0 1 1 0 3 3 2 2 3 3 2
 1 2 1 3 3 0 2 2 4 3 1 1 2 1 0 0 3 3 1 1 2 2 0 3 2 1 0 1 3 0 2 3 2 3 0 2 3
 1 2 0 2 2 2 1 0 2 3 2 1 2 2 1 1 2 1 2 2 2 3

In [28]:
# 예시: id와 예측된 클래스 저장
submission_df = pd.DataFrame({
    'idx': test_data['idx'],
    'class': predictions
})

display(submission_df)

# submission.csv로 저장
submission_df.to_csv('submission_kcbert_300_trainonly.csv', index=False)

,idx,class
0,t_000,1
1,t_001,2
2,t_002,2
3,t_003,3
4,t_004,3
...,...,...
495,t_495,2
496,t_496,2
497,t_497,1
498,t_498,2
